In [1]:
from sympy.logic.boolalg import And, Or, Not, Implies, Equivalent
import numpy as np
from sympy import symbols, bool_map
from sympy.logic.inference import satisfiable
from itertools import combinations
from belief_agent import BeliefBase
from utils import generate_all_subsets

**KB: Robert does well in the exam if and only if he is prepared or lucky.**
Robert does not do well the exam.

**Question: KB |= φ?**  **(KB entails phi)**

1. (-r $\vee$ p $\vee$ s) $\wedge$ (p $\wedge$ r) $\wedge$ (s $\wedge$ r) $\wedge$ $\neg$ r, formalisation of KB as CNF.

2.(-r $\vee$ p $\vee$ s) $\wedge$ (p $\wedge$ r) $\wedge$ (s $\wedge$ r) $\wedge$ r $\wedge$ p, CNF for contradiction.

3. clauses:= (-r $\vee$ p $\vee$ s), (p $\wedge$ r), (s $\wedge$ r), r, p.

4. clauses:=clauses U {r} from resolving (¬p $\vee$ r) with p.

5. clauses:=clauses U {1} from resolving r with $\neg$ r.

6. Result: empty clause, so KB |= φ.

In [2]:
belief_base = BeliefBase('q p')
symb = belief_base.symbols
q, p = symb

belief_base.expand(p, 1)
belief_base.expand(q, 2)
belief_base.expand(Implies(p, q), 3)
print("Belief Base:", belief_base)

Belief Base: <belief_agent.BeliefBase object at 0x119175910>


In [3]:
def generate_subsets(belief_base, n):
    return list(combinations(belief_base, n))

def generate_all_subsets(belief_base):
    subsets = []
    for n in range(1, len(belief_base)):
        subsets.extend(generate_subsets(belief_base, n))
    return subsets

In [23]:
from sympy.logic.boolalg import And, Or, Not, Implies, Equivalent, to_cnf
from sympy import symbols
from sympy.logic.inference import satisfiable
from utils import generate_all_subsets

class BeliefBase:
    def __init__(self, atomic_symbols: str):
        self.beliefs = set()
        self.beliefs_priorities = set()
        self.symbols = symbols(atomic_symbols)
        self.priorities = {}

    def expand(self, belief, priority):
        cnf_belief = to_cnf(belief)
        self.beliefs.add((cnf_belief))
        self.beliefs_priorities.add((cnf_belief, priority))
        self.priorities[cnf_belief] = priority

    def remove(self, belief, belief_set=None):
        cnf_belief = to_cnf(belief)
        if belief_set is None:
            if cnf_belief in self.beliefs:
                self.beliefs.remove(cnf_belief)
        else:
            if cnf_belief in belief_set:
                belief_set.remove(cnf_belief)

    def contract(self, extract_belief):
        sorted_beliefs = sorted(self.beliefs_priorities, key=lambda x: (-x[1]))
        
        # Might be and IF instead of while
        while self.check_entailment(extract_belief):
            entailed_formulas = []
            all_subsets = generate_all_subsets(self.beliefs)
            for subset in all_subsets:
                if self.check_entailment(formula=extract_belief, belief_base=set(subset)):
                    entailed_formulas.append(subset)
            
            lowest_priority = - np.inf
            formula_to_remove = None
            for sub_belief_set in entailed_formulas:
                for formula in sub_belief_set:
                    if self.priorities[formula] > lowest_priority:
                        lowest_priority = self.priorities[formula]
                        formula_to_remove = formula
            self.remove(formula_to_remove)
                        

    def revise(self, belief):
        # Levi's identity
        self.contract(Not(belief))
        self.expand(belief)

    def check_entailment(self, formula, belief_base=None):
        if belief_base is None:
            cnf_formula = to_cnf(formula)
            belief_base_cnf = to_cnf(self.beliefs)
            new_set = And(*belief_base_cnf, Not(cnf_formula))
            return not satisfiable(new_set)
        else:
            cnf_formula = to_cnf(formula)
            belief_base_cnf = to_cnf(belief_base)
            new_set = And(*belief_base_cnf, Not(cnf_formula))
            return not satisfiable(new_set)         
        
# Test
if __name__ == "__main__":
    belief_base = BeliefBase('p q')  
    p, q = belief_base.symbols
    # Expand belief base
    belief_base.expand(p, 3)
    belief_base.expand(q, 2)
    belief_base.expand(Implies(p, q), 1)
    print(belief_base.beliefs)
    
    belief_base.contract(q)
    print("\nFinal belief base:")
    print(belief_base.beliefs) 

{p, q, q | ~p}

Final belief base:
{q | ~p}


In [26]:
belief_base = BeliefBase('p q')  
p, q = belief_base.symbols
belief_base.expand(p, 3)
belief_base.expand(q, 2)
belief_base.expand(Implies(p, q), 1)

belief_base.contract(q)
print("\nFinal belief base:")
print(belief_base.beliefs) 


Final belief base:
{q | ~p}


In [14]:
type(belief_base.beliefs)

set

In [15]:
belief_base.check_entailment(formula=And(p,q), belief_base=belief_base.beliefs)

TypeError: expecting bool or Boolean, not `{p, q, p | q, q | ~p}`.